In [1]:
import cv2
import time
import math

In [2]:
cap = cv2.VideoCapture("car.mp4")  # read the video


#Define Width & height
width=1340
height=700


# Coordinates for first line ax 
ax1 = 1
ax2 = 1
ay1 = 0
ay2 = height

# Coordinates for line bx
bx1 = width-1
bx2 = width-1
by1 = 0
by2 = height


#Define the saved video
fps = cap.get(cv2.CAP_PROP_FPS)# frames per second (FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')# codec for video,resolution
out = cv2.VideoWriter('Speed_car_detention for each distance.mp4', fourcc, fps, (width, height))





# Function to check intersection of rectangle of cars &lines
def check_intersection(x, y, w, h, line_x1, line_y1, line_x2, line_y2):
   
    if (line_x1 <= x <= line_x2 and line_y1 <= y <= line_y2) or \
       (line_x1 <= x + w <= line_x2 and line_y1 <= y <= line_y2) or \
       (line_x1 <= x <= line_x2 and line_y1 <= y + h <= line_y2) or \
       (line_x1 <= x + w <= line_x2 and line_y1 <= y + h <= line_y2):
        return True
    elif ((x < line_x1 < x + w) or (x < line_x2 < x + w)) and ((line_y1 < y < line_y2) or (line_y1 < y + h < line_y2)):
        return True
    else:
        return False
    
    
    
    
# Function to calc speed
def calc_speed(distance, time):
    try:
         speed = (distance / 1000) / (time / 3600)  # Convert distance to kilometers and time to hours
        #divide to 1000 for kilometer ,3600 to hours (60*60) 60 seconds in a minute , 60 minutes in a hour
         return speed
    except ZeroDivisionError: #evoid erros
        print("Can't divide by zero")


        
        
# Initialize background subtractor
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)



# Object counting variables
object_count = 0
object_counts = {}
object_ids = {}

start_time = time.time()  # Start time for video processing


while True:
    ret, frame = cap.read()  # Read frame 
    if not ret:  # Stop running if frame is False (video ended)
        break

   
    frame = cv2.resize(frame, (width, height)) # Resize the frame
    mask = object_detector.apply(frame) # Apply background subtraction 
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)# Threshold the mask
    mask = cv2.erode(mask, None, iterations=1)# shrink unnecessary parts
    mask = cv2.dilate(mask, None, iterations=40)# Dilate important parts (cars)
    mask = cv2.erode(mask, None, iterations=12)# shrink again to miniize sz of cars

    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)# Find contours in the mask

   
    current_ids = []

    for cnt in contours:
        area = cv2.contourArea(cnt)  # Calculate contour area

        if area > 8000:  # Skip small contours
            x, y, w, h = cv2.boundingRect(cnt)

           
            x_center = x + w // 2
            y_center = y + h // 2

            
            object_id = None
            for oid, (cx, cy, _, _) in object_ids.items():
                distance = ((x_center - cx) ** 2 + (y_center - cy) ** 2) ** 0.5  # Euclidean distance
                if distance < 150:  # Threshold for considering objects as same
                    object_id = oid
                    break

            
            if object_id is None: # check if the object is new
                object_id = object_count
                object_count += 1
                object_counts[object_id] = -1

           
            object_counts[object_id] += 1

           
            current_ids.append(object_id)
            object_ids[object_id] = (x_center, y_center, w, h)

            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)# Draw rectangle around object
            text = f"{object_id}"
            cv2.putText(frame, text, (x + 10, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) #write count of ach car

            # Draw lines
            cv2.line(frame, (ax1, ay1), (ax2, ay2), (255, 0, 0), 2)# first one
            cv2.line(frame, (bx1, by1), (bx2, by2), (255, 0, 0), 2)# second one
            
            
            distance = math.sqrt((x_center - 0) ** 2 + (y_center - 0) ** 2)# Calculate distance from object to line ax         
       
            current_time = time.time() - start_time# Calculate diffence of  time
            speed = calc_speed(distance, current_time) # Calculate speed

           
            speed_text = f"Speed: {speed:.1f} km/h"
            cv2.putText(frame, speed_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2) # Display speed on the frame

            # Draw rectangle
            if check_intersection(x, y, w, h, ax1, ay1, ax2, ay2):  # Check if rectangle intersects with line a
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color
            elif check_intersection(x, y, w, h, bx1, by1, bx2, by2):  # Check if rectangle intersects with line b
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color
            else:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green color
                
                
    # Remove objects that are no longer visible
    removed_ids = set(object_ids.keys()) - set(current_ids)
    for oid in removed_ids:
        object_ids.pop(oid)

    
    out.write(frame)# save the output vid

   
    cv2.imshow("Detection of speed for each distances", frame)
#     cv2.imshow("Mask", mask)

 
    if cv2.waitKey(150) == 27:  # press 'esc' button to stop
        break


cap.release()
out.release()
cv2.destroyAllWindows()


### Wish u luck
### Esraa Meslam